In [ ]:
%matplotlib inline

### imports for sedlib

In [ ]:
from sedlib import SED, Filter, BolometricCorrection

### helper libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
from IPython.display import IFrame, HTML
from astropy import units as u
from astropy.coordinates import SkyCoord

from dust_extinction.parameter_averages import G23

warnings.simplefilter("ignore")

# 1. Filter Class

The **`Filter`** class provides access to transmission curves from the `SVO Filter Profile Service`, which contains thousands of astronomical filters used by various surveys and instruments.

In [ ]:
IFrame('https://svo2.cab.inta-csic.es/svo/theory/fps/', 1280, 420)

## 1.1. If the *"filter_id"* identifier is not known exactly

In [ ]:
f = Filter()
f.search('*TESS*')

## 1.2. If svo *"filter_id"* is known

In [ ]:
f = Filter('TESS/TESS.Red')
f

In [ ]:
# transmission curve
f.plot()

In [ ]:
f.WavelengthEff

In [ ]:
# Zero Point
f.ZeroPoint

## 1.3 Using Filters for Calculations

Filters can interpolate transmission values at any wavelength:

In [ ]:
# interpolated tramissance value for any value
f(5500 * u.AA)

In [ ]:
# or interpolated tramissance values for an array
ws = np.arange(5000, 5500, 100) * u.AA
f(ws)

### Converting Between Magnitudes and Fluxes

In [ ]:
# Convert magnitude to flux density using filter's zero point
f.mag_to_flux(12)

In [ ]:
# Convert back to magnitude
f.flux_to_mag(_)

## 1.4. Creating a custom filter transmission curve

In [ ]:
# Create a neutral density filter (uniform 10% transmission)
nd = Filter()

In [ ]:
# Define wavelength range and transmission
wavelength = np.arange(3000, 9000) * u.AA
flux = np.full(shape=wavelength.shape, fill_value=0.1)

In [ ]:
# Create the filter
nd.from_data(name='ND1.0', wavelength=wavelength, transmission=flux)

In [ ]:
nd(5500 * u.AA)

# 2. SED Class

## 2.1 SED Class Initialization

The SED class allows users to initialize with a variety of inputs:

- Name: Identifier for the object (e.g., "Algol").

- Coordinates: RA/Dec values, with support for different coordinate frames like ICRS.

- Search Radius: Defines the sky region to query databases, defaulting to 1 arcsecond.


Caching:

- Reduces unnecessary web queries to Simbad, VizieR, Gaia etc..

- Prevents exceeding rate limits or being blacklisted by these services during iterative analyses.

### 2.1.1 From SkyCoord object

```python
# Method 1: Using SkyCoord object
sed = SED(
    coord=SkyCoord(
        "16 17 59.2", "+51 23 31.53",
        unit=(u.hourangle, u.degree),
        frame='icrs'
    ),
    search_radius=1*u.arcsec
)
```

### 2.1.2 If RA and Dec are known


```python
# Method 2: Using RA and Dec strings
sed = SED(
    ra="16 17 59.2",
    dec="+51 23 31.53",
    search_radius=1*u.arcsec
)
```

### 2.1.3 If star identifier is known
The most convenient method when you know the object name:

In [ ]:
# Initialize with a Gaia identifier
sed = SED(
    name="Gaia DR3 3411009597090160768"
)

The SED class automatically queries multiple astronomical databases (Simbad, VizieR, Gaia) to collect all available photometric measurements for your target. This automated data collection is crucial for big data astronomy where manual data gathering is impractical.

---

#### Understanding the Caching System

The SEDLIB employs an intelligent caching mechanism that provides several critical benefits:

- **Bandwidth Conservation**: Prevents redundant queries to external databases, significantly reducing data transfer requirements
- **API Rate Limit Protection**: Minimizes the risk of being blacklisted by web services due to excessive requests—a common issue in iterative analysis workflows
- **Performance Optimization**: Reduces data retrieval overhead, enabling researchers to perform repeated analyses on the same dataset efficiently
- **Reliability**: Ensures consistent access to data even when external services experience temporary unavailability

This caching system is particularly valuable for large-scale studies involving thousands of objects, where network efficiency directly impacts computational feasibility.

## 2.2 Data Handling: The Catalog Class
Once initialized, you can examine the collected photometric data through the integrated Catalog class, which serves as the organizational backbone for photometric data management:

In [ ]:
# View the raw catalog table (stored as Astropy Table)
sed.catalog.table

The catalog is structured as an **Astropy Table** with the following organizational features:

- **Structured Storage**: Columns for RA, Dec, wavelength, flux, flux error, and filter identifications
- **Unit Consistency**: Ensures all data maintains compatible units for analysis using Astropy's unit system
- **Metadata Preservation**: Retains source catalog information and measurement provenance

This table contains all photometric measurements found for your object, including:
- Magnitudes and fluxes in different filter systems
- Photometric uncertainties and error estimates
- Source catalog identifications (Gaia, 2MASS, WISE, etc.)
- Filter transmission curve metadata and effective wavelengths

### 2.2.1 Data Cleaning and Preparation

#### Finding and Removing Missing Data

In [ ]:
# Find rows with missing filter information
sed.catalog.find_missing_data_rows('filter')

In [ ]:
# Remove these rows
sed.catalog.delete_missing_data_rows('filter')

In [ ]:
# sed.catalog.delete_rows({'eflux': '==0'})

In [ ]:
# sed.catalog.delete_missing_data_rows('eflux')

In [ ]:
# sed.catalog.filter_outliers(sigma_threshold=3.0, over_write=True)

In [ ]:
# Combine fluxes using mean values
sed.catalog.combine_fluxes(method='mean', overwrite=True)

In [ ]:
# Plot the SED with a blackbody fit
sed.plot(with_blackbody=True)

### 2.2.2 SQL-Like Data Querying Interface

In [ ]:
# Find measurements in specific wavelength range
sed.catalog.sql_query(
    "SELECT * FROM catalog WHERE wavelength between 0.5 and 0.6"
)

In [ ]:
# Find all Johnson filter measurements
sed.catalog.sql_query(
    "SELECT * FROM catalog WHERE vizier_filter LIKE '%Johnson%'"
)

## 2.3 Stellar Radius Estimation: Blackbody Modeling and Statistical Analysis
One of sedlib's key capabilities is estimating stellar radii through blackbody model fitting to multi-wavelength photometric data. The framework implements the fundamental relationship:

**F<sub>ν</sub> = B<sub>ν</sub>(T) · π · (R²/d²)**

where:
- **F<sub>ν</sub>** is the observed flux density
- **B<sub>ν</sub>(T)** is the Planck function at temperature T
- **R** is the stellar radius
- **d** is the distance to the object

This approach enables determination of stellar radii even when direct interferometric measurements are unavailable, making it particularly valuable for large-scale surveys.

### 2.3.1 Grid Search
The grid search method systematically tests different radius values to find the best fit:

In [ ]:
# Estimate radius using grid search
radius_result = sed.estimate_radius(
    method='grid',
    accept=False,      # Don't automatically accept the result
    verbose=True,      # Show progress information
    grid_min=0.1,      # Minimum radius to test (solar radii)
    grid_max=50.0,     # Maximum radius to test
    plot_chi2=True     # Show the chi-squared plot
)

### 2.3.2 Monte Carlo

The Monte Carlo method:
- Accounts for measurement uncertainties
- <ins>Provides realistic error estimates</ins>

In [ ]:
# Estimate radius with uncertainties using Monte Carlo
radius_result = sed.estimate_radius(
    method='mc',
    accept=True,       # Accept and store the result
    verbose=True,      # # Show progress information
    n_samples=1000,    # Number of Monte Carlo samples
    corner_plot=True,  # Show parameter correlation plots
    n_jobs=-1          # Number of parallel jobs to use (-1 uses all processors)  
)

## 2.4 Interstellar Extinction Estimation and Correction
#### Estimating The E(B-V)

Interstellar dust fundamentally alters observed stellar photometry through wavelength-dependent absorption and scattering. The framework implements modern extinction laws (such as G23 with R<sub>V</sub>=3.1) to estimate the color excess E(B-V) and apply appropriate corrections to recover intrinsic stellar parameters.

The extinction correction is critical for accurate parameter determination, particularly for distant stars where extinction effects can significantly obscure fundamental stellar properties. The framework's implementation accounts for both the magnitude of extinction and its wavelength dependence.

### 2.4.1 Minimize
This method uses optimization to find the extinction that best matches the data:

In [ ]:
# # Estimate extinction using minimization
# sed.estimate_ebv(
#     method='minimize',
#     accept=True,
#     verbose=True,
#     num_points=1000,    # Grid resolution for initial search
# )

### 2.4.2 Monte Carlo

In [ ]:
# Estimate extinction with uncertainties using Monte Carlo
sed.estimate_ebv(
    method='mc',
    accept=True,
    verbose=True,
    n_samples=1000,    # Number of samples for Monte Carlo method
    plot_corner=True,
    n_jobs=-1,         # Number of parallel jobs to use (-1 uses all processors)  
    show_progress=True
)

### 2.4.3 Plot

In [ ]:
# Plot SED with extinction correction
sed.plot(
    with_blackbody=True,
    with_extinction=True,
)

## 2.5 Bolometric Corrections and Radius Determination

Bolometric corrections are fundamental for converting observed magnitudes to bolometric magnitudes, accounting for stellar flux outside the observed wavelength range. This process is essential for determining accurate stellar luminosities, particularly for stars with significant UV or IR emission not captured by optical photometry.

### 2.5.1 Preparations

In [ ]:
# Compute extinction at each wavelength
sed.compute_A_lambda()

In [ ]:
# Compute absolute magnitudes
sed.compute_absolute_magnitudes()

In [ ]:
# catalog
sed.catalog.table

### 2.5.2 Radius Estimation with BC Correction

In [ ]:
# Create bolometric correction object
bc = BolometricCorrection(sed, accept_radius=True)

In [ ]:
# Run the correction
_ = bc.run(verbose=True)

In [ ]:
# detailed bc information
bc._result

## 2.6 Automated Pipelines for Large-Scale Analysis

Sedlib provides sophisticated pipeline functions designed for big data astronomy applications. The framework has been tested on datasets containing approximately **32,900** single stars and is designed to scale efficiently for survey-level analyses involving millions of objects.

The pipeline architecture leverages parallelized computation using `joblib`, enabling simultaneous processing of thousands of stars while maintaining computational efficiency and memory management.

### 2.6.1 Quick Analysis

For rapid results with default settings:
- Data collection and cleaning
- Radius estimation
- Extinction estimation  
- Bolometric corrections
- Result visualization
- Save project

In [ ]:
# Create SED object and run complete analysis
sed = SED("Gaia DR3 46765188763683200", info=False)
sed.run()

### 2.6.2 Detailed Analysis
For full control over the analysis:

In [ ]:
# Define detailed pipeline parameters
paramters = {
    # High-level flags for each pipeline stage
    'clean_data': True,
    'combine_fluxes': False,
    'filter_outliers': False,
    'estimate_radius': False,
    'estimate_extinction': True,
    'compute_bolometric': True,
    # Configuration dictionaries for each stage
    'data_cleaning_config': {
        'delete_missing_data_columns': 'filter'
    },
    'flux_combination_config': {
        'method': 'median'
    },
    'outlier_filtering_config': {
        'sigma_threshold': 3.0
    },
    'radius_estimation_config': {
        'method': 'mc',
        'accept': True,
        'n_samples': 5000,
        'corner_plot': True,
        'grid_min': 0.1,
        'grid_max': 50.0,
        'grid_points': 10000,
        'refine_window': 0.3,
        'refine_steps': 10000,
        'plot_chi2': True,
        'show_progress': True
    },
    'extinction_estimation_config': {
        'method': 'mc',
        'model': 'blackbody',
        'ext_model': G23(Rv=3.1),
        'accept': True,
        'n_samples': 5000,
        'corner_plot': True,
        'ebv_range': (0.0, 10.0),
        'ebv_initial': 0.1,
        'num_points': 10000,
        'optimization_method': 'L-BFGS-B',
        'show_progress': True
    },
    'bolometric_correction_config': {
        'accept_radius': True
    },
    # General settings
    'silent': False,
    'verbose': True,
    'plot_results': True,
    'continue_on_error': True,
    'save': True,
    'save_path': None,
    'save_compressed': True,
    'return_results': False,
}

In [ ]:
# Run the customized pipeline
sed.run(**paramters)

## 2.7 Project Management

### 2.7.1 Save Project

In [ ]:
# Save complete analysis to file and automatically zip it
sed.save('/Users/oguzhan/example.sedlib')

### 2.7.2 Load Project

In [ ]:
# Load a saved project
sed = SED.load('/Users/oguzhan/example.sedlib.zip')

In [ ]:
# Plot Monte Carlo radius results
sed.plot_results(radius_mc=True)

In [ ]:
# Plot extinction results
sed.plot_results(ebv_mc=True)

In [ ]:
# Interactive SED plot
sed.plot(
    with_blackbody=True,
    with_extinction=True,
    with_outliers=True,
    interactive=True
)